In [48]:
###basic csv file creation
import csv

with open('persons.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Name', 'Profession'])
    writer.writerow(['Derek', 'Software Developer'])
    writer.writerow(['Steve', 'Software Developer'])
    writer.writerow(['Paul', 'Manager'])

In [15]:
### writing multiple rows from a python list
import csv
row_list = [["SN", "Name", "Contribution"],
            [1, "Linus Torvalds", "Linux Kernel"],
            [2, "Tim Berners-Lee", "World Wide Web"],
            [3, "Guido van Rossum", "Python Programming"]]
with open('protagonist.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(row_list)

In [ ]:
### importing midi
import mido ###why doesn't this effing work?
from mido import MidiFile
mid = MidiFile('test1.mid')

### printing midi data;;; be advised that in MIDI format a note_on messsage with a velocity of 0 is equivalent to a note_off message
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(str(msg))

#how to see how many tracks in a MidiFile
mid.print_tracks(meta_only=True)

# mido.ticks2second(3840,mid.ticks_per_beat,)

In [17]:
### Getting only note_on messages
from mido import MidiFile
mid = MidiFile('ockIntroitus.mid')

def test(example):
    for i, track in enumerate(example.tracks):
        print('Track {}: {}'.format(i, track.name))
        for msg in track:
            if msg.type == 'note_on':
                if msg.velocity == 0:
                    print('note_off ' + str(msg)[8:]) # this is for MIDI tracks that implement note_offs as zero velocity note_ons ; standardizes to using note_offs
                else: print(str(msg))



In [10]:
### \\\ --- tools to generate DynamicNodeList from a midi file --- /// ###

from mido import MidiFile
import itertools
import csv
mid = MidiFile('test1.mid')

def midi2list(midi_file): # gets only the note_on and note_off messages, creates a sublist for each, and breaks their parameters into separate strings
    for i, track in enumerate(midi_file.tracks):
        extraction = [ list(str(msg).split(" ")) for msg in track if msg.type == 'note_on' or msg.type == 'note_off' ]
        yield [ 'Track {} {}'.format(i, track.name), list(extraction) ]

step1 = list(midi2list(mid))
#print(step1)

def get_full(something): # filters out empty message lists (usually the meta message list is stored in track 0, so it's empty from the midi2list function filtering only note_on/note_off messages)
    return [ i for i in something if len(i[1]) != 0]

step2 = get_full(step1)
# print(step2)

def numberer(message_list): # adds number to beginning of each sub-list, which will be used in .csv creation to indicate row number (turns out this is not needed, I misunderstood how csv writer worked...)
    for entry in message_list:
        counter = 1
        for message in entry[1]:
            message.insert(0, counter)
            counter += 1
    return message_list

step3 = numberer(step2)
# print(step3)

def standardizer(numbed_list): # tests for note_on messages with velocity = 0; changes them to note_offs
    for entry in numbed_list:
        for message in entry[1]:
            if message[4] == 'velocity=0':
                 message[1] = 'note_off'
    return numbed_list

step4 = standardizer(step3)
# print(step4)

combined_func = standardizer(numberer(get_full(list(midi2list(mid)))))
# print(combined_func)

def abs_onsets(pl): # gets just the onset delta-times and converts them to an int list of absolute times
    for entry in pl:
        deltas = [ int( message[5].partition('=')[2]) for message in entry[1] ]
        yield itertools.accumulate(deltas)

onsetz = list(abs_onsets(combined_func))

def lister(blah): # this is just dealing with the yield iterator
    return [ list(item) for item in blah  ]

list_onsetz = lister(onsetz)
# print(list_onsetz)

def final_format(combined_func, onsets): # finally putting everything together
    for i, entry in enumerate(combined_func):
        for j in range(0,len(entry[1]), 2): #we want to step through 'combined_func' by twos, to get the start and end time of a single node from a note_on message and the note_off immediately following
            yield [ onsets[i][j], onsets[i][j+1], combined_func[i][1][j][3].partition('=')[2], 'FALSE', 'FALSE',  onsets[i][j+1] - onsets[i][j] ]

node_partition_sizes = [len(entry[1]) // 2 for entry in combined_func] # but the yield doesn't distinguish between tracks, so we have to make this list for how many messages each track contains
# print(node_partition_sizes)

ff = final_format(combined_func, list_onsetz)
# print(list(ff))

partitioned_list = [list(itertools.islice(ff, elem)) for elem in node_partition_sizes] #islice takes a generator for its first argument! i.e. the yield from 'final_format'. And we use 'node_partition_sizes' to tell islice how to group messages from the yield generator.
# print(partitioned_list)

def csv_dynamic_node_maker(partitioned_list): #making a .csv file out of each track's messages in 'partitioned_list' and saving it
    for i, entry in enumerate(partitioned_list):
        entry = [['onset', 'terminus', 'vertex.id', 'onset.censored', 'terminus.censored', 'duration']] + entry # column header labels
        title = i + 1 # titling each .csv file by its track's index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_dynamic_nodes.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)

csv_dynamic_node_maker(partitioned_list)

def edge_converter(partitioned_list):
    for track in partitioned_list:
        for j in range(len(track) - 1): #we need to stop this iterative process before we reach the last message so that j doesn't go out of range
            tail = track[j][2]
            head = track[j+1][2]
            onset = track[j+1][0]
            terminus = track[j+1][1]
            duration = terminus - onset
            edge_id = tail + head
            yield [onset, terminus, tail, head, 'FALSE', 'FALSE', duration, edge_id ]

edge_conversion = edge_converter(partitioned_list)
# print(list(edge_conversion))

edge_partition_sizes = [i-1 for i in node_partition_sizes]# num(edges) is always num(nodes) - 1, in each track;
# print(edge_partition_sizes)

partitioned_edge_conversion = [list(itertools.islice(edge_conversion, elem)) for elem in edge_partition_sizes] # like before the yield generator needs to be told how to group results into tracks.
# print(partitioned_edge_conversion)

def edge_id_maker(partitioned_edge_conversion): #generates a list of unordered sets containing the unique node_pair strings for each track
    for track in partitioned_edge_conversion:
        node_pair_set = set()
        for msg in track:
            node_pair_set.add(msg[7])
        yield node_pair_set

edge_list = list(edge_id_maker(partitioned_edge_conversion))
# print(edge_list)

def edge_id_dict_maker(edge_list): # list of dictionaries containing node_pair strings as keys and ints as values
    return [{key:count for count, key in enumerate(edge, 1)} for edge in edge_list]

edge_id_dict_list = edge_id_dict_maker(edge_list)
# print(edge_id_dict_list)

def edge_id_converter(partitioned_edge_conversion, edge_id_dict_list): # replacing each unique node_pair string with its dict value (per track)
    dummy = partitioned_edge_conversion.copy()
    for i, track in enumerate(dummy):
        for msg in track:
            msg.append(edge_id_dict_list[i][msg[7]])
            msg.pop(7)
    return dummy

edge_id_conversion = edge_id_converter(partitioned_edge_conversion, edge_id_dict_list)
# print(edge_id_conversion)

def csv_dynamic_edge_maker(edge_id_conversion):
    for i, entry in enumerate(edge_id_conversion):
        entry = [['onset', 'terminus', 'tail', 'head', 'onset.censored', 'terminus.censored', 'duration', 'edge.id']] + entry # column header labels
        title = i + 1 # titling each .csv file by its index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_dynamic_edges.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)

csv_dynamic_edge_maker(edge_id_conversion)

def static_edge_maker_conversion(edge_id_conversion): # getting only the tail and head info from the dynamic edge list
    for track in edge_id_conversion:
        yield [[msg[2], msg[3]] for msg in track ]

static_edge_list = list(static_edge_maker_conversion(edge_id_conversion))
# print(static_edge_list)

def csv_static_edge_maker(static_edge_list):
    for i, entry in enumerate(static_edge_list):
        entry = [['tail', 'head']] + entry #column header labels
        title = i + 1 # titling each .csv file by its index ( we filtered out MetaMessage track 0, so need to add 1 to index to accurately reflect original track number)
        with open('track_{}_static_edges.csv'.format(title), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(entry)

csv_static_edge_maker(static_edge_list)

[[[3840, 7679, '74', '74', 'FALSE', 'FALSE', 3839, 1], [7680, 11519, '74', '74', 'FALSE', 'FALSE', 3839, 1], [11520, 15359, '74', '74', 'FALSE', 'FALSE', 3839, 1]], [[3840, 7679, '71', '72', 'FALSE', 'FALSE', 3839, 2], [7680, 11519, '72', '72', 'FALSE', 'FALSE', 3839, 1], [11520, 15359, '72', '72', 'FALSE', 'FALSE', 3839, 1]], [[3840, 7679, '71', '71', 'FALSE', 'FALSE', 3839, 1], [7680, 11519, '71', '71', 'FALSE', 'FALSE', 3839, 1], [11520, 15359, '71', '71', 'FALSE', 'FALSE', 3839, 1]]]


In [73]:
### Creating .csv file for MIDI note names ###

chromatic_pitches = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']

def MIDI_pitch_mapping(chromatic_pitches):
    i = 0
    for j in range(24,128):
        note = chromatic_pitches[(j-24) % 12]
        if note == 'C':
            i += 1
        yield [ j, note + str(i) ]

shame = [[21, 'A0'], [22, 'A#0'], [23, 'B0']]

MIDI_map = shame + list(MIDI_pitch_mapping(chromatic_pitches))

def csv_vertex_attributes_maker(MIDI_map):
    alias = MIDI_map[:]
    alias.insert(0, ['vertex.id', 'note_name',]) # column header labels
    with open('MIDI_map.csv', 'w', newline='') as file: 
        writer = csv.writer(file)
        writer.writerows(alias)

csv_vertex_attributes_maker(MIDI_map)